In [1]:
from datetime import datetime

begin_time = datetime.now()  ##
print("Begin Time : {}".format(begin_time))
import pandas as pd, os, sys, glob, shutil
import pyarrow as pa
import pyarrow.parquet as pq

if not sys.warnoptions:
    import warnings

    warnings.simplefilter("ignore")  # Stop Pandas warnings


Begin Time : 2020-09-13 13:16:16.786620


In [2]:
if os.getlogin() == "WylieTimmerman":
    # Working Paths
    path_working = r"C:\OD\OneDrive - Foursquare ITP\Projects\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(r"C:\OD\OneDrive - Foursquare ITP\Projects\WMATA_AVL")
    path_sp = r"C:\Users\WylieTimmerman\Documents\projects_local\wmata_avl_local"
    path_source_data = os.path.join(path_sp,"data","00-raw")
    path_processed_data = os.path.join(path_sp, "data", "02-processed")
elif os.getlogin() == "abibeka":
    # Working Paths
    path_working = r"C:\Users\abibeka\OneDrive - Kittelson & Associates, Inc\Documents\Github\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(path_working)
    # Source data
    path_source_data = r"C:\Users\abibeka\OneDrive - Kittelson & Associates, Inc\Documents\WMATA-AVL\Data"
    gtfs_dir = os.path.join(path_source_data, "google_transit")
    # Processed data
    path_processed_data = os.path.join(path_source_data, "ProcessedData")
elif os.getlogin() == "E043868":
    # Working Paths
    path_working = r"C:\Users\e043868\OneDrive - WMATA\R Projects\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(r"C:\Users\e043868\OneDrive - WMATA\R Projects\WMATA_AVL")
    path_source_data = r"\\l-600730\RawNavArchive"
    path_sp = r"C:\Users\e043868\Documents\RawNav"
    path_processed_data = os.path.join(path_sp, "data", "02-processed")

elif os.getlogin() == "E048374":
    # Working Paths
    path_working = r"C:\Users\E048374\OneDrive - WMATA\rawnav_rachel_fork\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(r"C:\Users\E048374\OneDrive - WMATA\rawnav_rachel_fork\WMATA_AVL")
    path_source_data = r"\\l-600730\RawNavArchive"
    path_sp = r"C:\Users\E048374\Documents\RawNav"
    path_processed_data = os.path.join(path_sp, "data", "02-processed")

else:
    raise FileNotFoundError("Define the path_working, path_source_data, gtfs_dir, \
                            ZippedFilesloc, and path_processed_data in a new elif block")

In [36]:
restrict_n = None

# List of Routes 
q_jump_route_list = ['52'] #update
# If desired, a subset of routes above or the entire list. Code will iterate on the analysis_routes list
analysis_routes = q_jump_route_list

run_inventory = True # inventory (or re-inventory files), otherwise reload saved inventory if available
run_existing = True # whether to redo outputs that currently exist or skip over them

# 1.3 Import User-Defined Package
############################################
import wmatarawnav as wr

execution_time = str(datetime.now() - begin_time).split('.')[0]
print("Run Time Section 1 Import Libraries and Set Global Parameters : {}".format(execution_time))

# 2 Identify Relevant Files for Analysis Routes
########################################################################################################################
begin_time = datetime.now()  ##

# Create a list of zipped rawnavfiles (ala 'rawnav06544171027.txt.zip') as 
# file_universe. 

#trying to get 52 - grabbing Nov 2019
zipped_files_dir_parent = os.path.join(path_source_data, "112019")
file_universe = glob.glob(os.path.join(zipped_files_dir_parent, "*.txt.zip"))
#instead of loading a subset list - just say top 50 below
# subset_file = os.path.join(path_working, "analysis", "bus-lane-analysis", "2020-07-busstate_file_IDs.csv")
# subset = pd.read_csv(subset_file)['date_log_format'].to_list()

Run Time Section 1 Import Libraries and Set Global Parameters : 0:01:06


## instead of using a list of files,I know vehicleid = 7164 went on route 52 - text search for that

In [37]:
file_universe_subset = [filename for filename in file_universe if '7164' in filename]

In [38]:
def Filter(string, substr): 
    return [str for str in string if
             any(sub in str for sub in substr)] 

# file_universe_subset = Filter(file_universe, subset)

In [39]:
begin_time = datetime.now()  ##
rawnav_inventory = wr.find_rawnav_routes(file_universe_subset, nmax=restrict_n, quiet=True)
execution_time = str(datetime.now() - begin_time).split('.')[0]
print("Run Time Getting rawnav inventory : {}".format(execution_time))

Run Time Getting rawnav inventory : 0:00:07


In [40]:
rawnav_inventory[rawnav_inventory.loc[:,'route']=='52']

,fullpath,filename,file_id,file_busid,taglist,line_num,line_num_next,route_pattern,route,pattern,tag_busid,tag_date,tag_time,tag_datetime,tag_starthour,wday,Unk1,mi_to_ft
2,\\l-600730\RawNavArchive\112019\rawnav07164191...,rawnav07164191103.txt,07164191103,7164,"19, 5201,7164,11/02/19,19:02:38,37143,05280",19,1729,5201,52,01,7164,2019-11-02,19:02:38,2019-11-02 19:02:38,19,Saturday,37143,05280
2,\\l-600730\RawNavArchive\112019\rawnav07164191...,rawnav07164191103.txt,07164191103,7164,"1729, 5204,7164,11/02/19,20:02:01,37143,05280",1729,3415,5204,52,04,7164,2019-11-02,20:02:01,2019-11-02 20:02:01,20,Saturday,37143,05280
2,\\l-600730\RawNavArchive\112019\rawnav07164191...,rawnav07164191103.txt,07164191103,7164,"3415, 5201,7164,11/02/19,20:57:00,37143,05280",3415,5020,5201,52,01,7164,2019-11-02,20:57:00,2019-11-02 20:57:00,20,Saturday,37143,05280
2,\\l-600730\RawNavArchive\112019\rawnav07164191...,rawnav07164191103.txt,07164191103,7164,"5020, 5204,7164,11/02/19,22:00:18,37143,05280",5020,6577,5204,52,04,7164,2019-11-02,22:00:18,2019-11-02 22:00:18,22,Saturday,37143,05280
6,\\l-600730\RawNavArchive\112019\rawnav07164191...,rawnav07164191107.txt,07164191107,7164,"5, 5202,7164,11/06/19,06:53:37,37454,05280",5,2002,5202,52,02,7164,2019-11-06,06:53:37,2019-11-06 06:53:37,6,Wednesday,37454,05280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,\\l-600730\RawNavArchive\112019\rawnav07164191...,rawnav07164191130.txt,07164191130,7164,"10955, 5204,7164,11/29/19,12:11:00,36718,05280",10955,12755,5204,52,04,7164,2019-11-29,12:11:00,2019-11-29 12:11:00,12,Friday,36718,05280
27,\\l-600730\RawNavArchive\112019\rawnav07164191...,rawnav07164191130.txt,07164191130,7164,"12755, 5201,7164,11/29/19,13:24:22,36718,05280",12755,14407,5201,52,01,7164,2019-11-29,13:24:22,2019-11-29 13:24:22,13,Friday,36718,05280
27,\\l-600730\RawNavArchive\112019\rawnav07164191...,rawnav07164191130.txt,07164191130,7164,"14407, 5204,7164,11/29/19,14:40:29,36718,05280",14407,15429,5204,52,04,7164,2019-11-29,14:40:29,2019-11-29 14:40:29,14,Friday,36718,05280
27,\\l-600730\RawNavArchive\112019\rawnav07164191...,rawnav07164191130.txt,07164191130,7164,"15429, 5201,7164,11/29/19,15:27:11,36718,05280",15429,16996,5201,52,01,7164,2019-11-29,15:27:11,2019-11-29 15:27:11,15,Friday,36718,05280


In [41]:
path_rawnav_inventory = os.path.join(path_processed_data,"rawnav_inventory.parquet")
shutil.rmtree(path_rawnav_inventory, ignore_errors=True) 
os.mkdir(path_rawnav_inventory)

In [42]:
rawnav_inventory.head()

,fullpath,filename,file_id,file_busid,taglist,line_num,line_num_next,route_pattern,route,pattern,tag_busid,tag_date,tag_time,tag_datetime,tag_starthour,wday,Unk1,mi_to_ft
0,\\l-600730\RawNavArchive\112019\rawnav07164191...,rawnav07164191101.txt,07164191101,7164,"5,PO05529,7164,10/31/19,21:06:33,37674,05280",5,136,PO05529,NaN,NaN,7164,2019-10-31,21:06:33,2019-10-31 21:06:33,21,Thursday,37674,05280
0,\\l-600730\RawNavArchive\112019\rawnav07164191...,rawnav07164191101.txt,07164191101,7164,"136, E401,7164,10/31/19,21:12:55,37674,05280",136,1925,E401,E4,01,7164,2019-10-31,21:12:55,2019-10-31 21:12:55,21,Thursday,37674,05280
0,\\l-600730\RawNavArchive\112019\rawnav07164191...,rawnav07164191101.txt,07164191101,7164,"1925, E403,7164,10/31/19,21:59:59,37674,05280",1925,3490,E403,E4,03,7164,2019-10-31,21:59:59,2019-10-31 21:59:59,21,Thursday,37674,05280
0,\\l-600730\RawNavArchive\112019\rawnav07164191...,rawnav07164191101.txt,07164191101,7164,"3490, E405,7164,10/31/19,22:38:08,37674,05280",3490,4728,E405,E4,05,7164,2019-10-31,22:38:08,2019-10-31 22:38:08,22,Thursday,37674,05280
0,\\l-600730\RawNavArchive\112019\rawnav07164191...,rawnav07164191101.txt,07164191101,7164,"4728, E404,7164,10/31/19,23:09:57,37674,05280",4728,5905,E404,E4,04,7164,2019-10-31,23:09:57,2019-10-31 23:09:57,23,Thursday,37674,05280


In [43]:
rawnav_inventory.route.unique()

array([nan, 'E4', '42', '43', '52', 'D2', 'L2', 'L1', 'N4', 'N2', 'N6',
       '54', 'TTT', '31', '33', 'D5', 'M4', 'D6'], dtype=object)

In [44]:
# Note: partitioning required, using filename avoids resorting of values, filename column
# will be sorted to end on reload however.
rawnav_inventory.to_parquet(
    path = path_rawnav_inventory,
    partition_cols = ['filename'],
    index = False
)

In [45]:
rawnav_inventory_filtered = (
    rawnav_inventory[
        rawnav_inventory
        .groupby('filename',sort = False)['route']
        .transform(lambda x: x.isin(analysis_routes).any())
    ]
)
   
rawnav_inventory_filtered = (
    rawnav_inventory_filtered
    .assign(line_num = lambda x: x.line_num.astype('int'))
)
    
if len(rawnav_inventory_filtered) == 0:
    raise Exception("No Analysis Routes found in file_universe")

execution_time = str(datetime.now() - begin_time).split('.')[0]
print("Run Time Section 2 Identify Relevant Files for Analysis Routes : {}".format(execution_time))

Run Time Section 2 Identify Relevant Files for Analysis Routes : 0:00:07


In [46]:
# 3 Load Raw RawNav data
########################################################################################################################
begin_time = datetime.now()
# data is loaded into a dictionary named by the ID
route_rawnav_tag_dict = {}

# Iterate over each file, skipping to the first row where data in our filtered inventory is found
# Rather than read run-by-run, we read the rest of the file, then filter to relevant routes later
rawnav_inv_filt_first = rawnav_inventory_filtered.groupby(['fullpath', 'filename']).line_num.min().reset_index()
rawnav_inventory_filtered_valid = rawnav_inventory_filtered

for index, row in rawnav_inv_filt_first.iterrows():
    tag_info_line_no = rawnav_inventory_filtered[rawnav_inventory_filtered['filename'] == row['filename']]
    reference = min(tag_info_line_no.line_num)
    # -1 refers to the fact that the tag line identifying the start of a run will be removed, such
    # that the second row associated with a run will become the first row of data. This helps to 
    # ensure that indices of the processed data will line up with values in the rawnav inventory
    tag_info_line_no.loc[:, "new_line_no"] = tag_info_line_no.line_num - reference - 1
    temp = wr.load_rawnav_data(
        zip_folder_path=row['fullpath'],
        skiprows=row['line_num'])

    if type(temp) != type(None):
        route_rawnav_tag_dict[row['filename']] = dict(RawData=temp, tagLineInfo=tag_info_line_no)
    else:
        remove_file = row['filename']  
        rawnav_inventory_filtered_valid  = rawnav_inventory_filtered_valid.query('filename!= @remove_file')

execution_time = str(datetime.now() - begin_time).split('.')[0]
print("Run Time Section 3 Load Raw RawNav Data : {}".format(execution_time))

Run Time Section 3 Load Raw RawNav Data : 0:00:00


In [47]:
# 4 Clean RawNav data
########################################################################################################################
begin_time = datetime.now()  ##

rawnav_data_dict = {}
summary_data_dict = {}

for key, datadict in route_rawnav_tag_dict.items():
    temp_dat = wr.clean_rawnav_data(
        data_dict=datadict,
        filename=key)

    rawnav_data_dict[key] = temp_dat['rawnavdata']
    summary_data_dict[key] = temp_dat['summary_data']     

route_rawnav_tag_dict = None

execution_time = str(datetime.now() - begin_time).split('.')[0]
print("Run Time Section 4 Clean RawNav Data : {}".format(execution_time))

Run Time Section 4 Clean RawNav Data : 0:00:17


In [48]:
# 5 Output
########################################################################################################################
begin_time = datetime.now()  
# 5.1 Output Summary Rawnav data
############################################

# Combine summary files, filter to analysis routes, convert col types once NAs removed
summary_rawnav = pd.concat(summary_data_dict.values())

summary_rawnav = summary_rawnav[summary_rawnav['route'].isin(analysis_routes)]
summary_rawnav = summary_rawnav.assign(
    route=lambda x: x.route.astype('str'),
    pattern=lambda x: x.pattern.astype('int32'))

# Remove duplicate runs
summary_rawnav = summary_rawnav[
    ~summary_rawnav.duplicated(['filename', 'index_run_start'], keep='last')]  

# Output Summary Files
path_summary_rawnav = os.path.join(path_processed_data,"rawnav_summary.parquet")

if not os.path.isdir(path_summary_rawnav):
    os.mkdir(path_summary_rawnav)
      
for analysis_route in analysis_routes:
    
    path_summary_route = os.path.join(path_summary_rawnav,"route={}".format(analysis_route))
    
    if (os.path.isdir(path_summary_route) and run_existing) or (not os.path.isdir(path_summary_route)): 
        shutil.rmtree(os.path.join(path_summary_route), ignore_errors=True) 
           
        summary_rawnav_fil = summary_rawnav.query('route == @analysis_route')
        
        table_summary = pa.Table.from_pandas(summary_rawnav_fil,
                                             schema = wr.rawnav_summary_schema())
        
        pq.write_to_dataset(table_summary, 
                            root_path=os.path.join(path_summary_rawnav),
                            partition_cols=['route','wday'])
    else:
        print('skipping summary output of {}'.format(analysis_route))

In [49]:
# 5.2 Output Processed Rawnav data
############################################
# Export Data
path_rawnav_data = os.path.join(path_processed_data, "rawnav_data.parquet")

if not os.path.isdir(path_rawnav_data):
    os.mkdir(path_rawnav_data)

for analysis_route in analysis_routes:
    
    path_rawnav_route = os.path.join(path_rawnav_data,"route={}".format(analysis_route))
    
    if (os.path.isdir(path_rawnav_route) and run_existing) or (not os.path.isdir(path_rawnav_route)): 
        shutil.rmtree(os.path.join(path_rawnav_route), ignore_errors=True) 
    
        # Merge Cleaned Rawnav Files Containing The Analysis Route
        out_rawnav_dat = wr.subset_rawnav_run(
            rawnav_data_dict_=rawnav_data_dict,
            rawnav_inventory_filtered_valid_=rawnav_inventory_filtered_valid,
            analysis_routes_=analysis_route)
        
        if out_rawnav_dat.shape == (0, 0):
            continue
        
        # Join Additional Identifying Information
        temp = summary_rawnav.query('route == @analysis_route')\
            [['filename', 'index_run_start', 'wday', 'start_date_time']]
                
        out_rawnav_dat = out_rawnav_dat.merge(temp, 
                                              on=['filename', 'index_run_start'], 
                                              how='left')
    
        assert (out_rawnav_dat.groupby(['filename', 'index_run_start', 'index_loc'])['index_loc'].
                count().values.max() == 1)
        
        # Column conversion after missing values removed
        out_rawnav_dat = out_rawnav_dat.assign(
            route=lambda x: x.route.astype('str'),
            #should be okay as int32 if everything goes to plan, but for safety will keep as double
            # and convert pattern later
            pattern=lambda x: x.pattern.astype('double')) 
    
        # Output    
        shutil.rmtree(os.path.join(path_rawnav_data,"route={}".format(analysis_route)), ignore_errors=True) 
                
        table = pa.Table.from_pandas(out_rawnav_dat,
                                     schema = wr.rawnav_data_schema())
        
        pq.write_to_dataset(table, 
                            root_path=os.path.join(path_rawnav_data),
                            partition_cols=['route','wday'])
    else:
        print('skipping output of {}'.format(analysis_route))

execution_time = str(datetime.now() - begin_time).split('.')[0]
print("Run Time Section 5 Output : {}".format(execution_time))
end_time = datetime.now()
print("End Time : {}".format(end_time))

Run Time Section 5 Output : 0:00:04
End Time : 2020-09-13 13:43:36.825422
